In [ ]:
import carla
import random
import time
import math

# Connessione al simulatore CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Ottenere il mondo di simulazione
world = client.get_world()

# Selezioniamo un veicolo dal mondo o creiamone uno
blueprint = random.choice(world.get_blueprint_library().filter('vehicle'))
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(blueprint, spawn_point)

# Aggiungi una telecamera per monitorare il comportamento
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.5))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Funzione per seguire la curva
def follow_curve(vehicle, curve_points, speed=30):
    # Impostiamo la velocità di base
    throttle = 0.5
    brake = 0.0
    steering = 0.0
    
    # Iteriamo sui punti della curva
    for point in curve_points:
        # Calcoliamo la direzione tra il veicolo e il punto successivo
        dx = point[0] - vehicle.get_location().x
        dy = point[1] - vehicle.get_location().y
        angle_to_target = math.atan2(dy, dx)  # Angolo tra il veicolo e il punto
        
        # Otteniamo la differenza tra l'angolo corrente e l'angolo da percorrere
        angle_difference = angle_to_target - vehicle.get_transform().rotation.yaw
        if angle_difference > math.pi:
            angle_difference -= 2 * math.pi
        elif angle_difference < -math.pi:
            angle_difference += 2 * math.pi
        
        # Impostiamo la sterzata in base all'angolo
        steering = max(-1.0, min(1.0, angle_difference / math.pi))
        
        # Impostiamo la velocità (accelera o frena)
        if abs(angle_difference) < 0.1:  # Se l'angolo è molto piccolo, vai dritto
            throttle = 0.7
        else:  # Altrimenti riduci la velocità
            throttle = 0.3
        
        # Applicare il controllo del veicolo
        vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steering, brake=brake))
        
        # Aggiorna il mondo e attendi un po'
        world.tick()
        time.sleep(0.1)

# Definiamo una curva (serie di punti)
curve_points = [
    (50, 50),
    (60, 60),
    (70, 50),
    (80, 60),
    (90, 50)
]

# Fai seguire al veicolo la curva
follow_curve(vehicle, curve_points)

# Dopo aver fatto seguire la curva al veicolo, puoi fermarlo
vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))

# Chiudere il simulatore
camera.destroy()
vehicle.destroy()
